# 7.6 循环神经网络的简洁实现
本节将展示如何使用深度学习框架的高级API提供的函数更有效地实现相同的语言模型。
我们仍然从读取时光机器数据集开始。

In [19]:
import torch
from torch import nn
from torch.nn import functional as F
from d2l import torch as d2l

batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)

## 7.6.1 定义模型
高级API提供了循环神经网络的实现。
我们构造一个具有256个隐藏单元的单隐藏层的循环神经网络层`rnn_layer`。
事实上，我们还没有讨论多层循环神经网络的意义。
现在仅需要将多层理解为一层循环神经网络的输出被用作下一层循环神经网络的输入就足够了。

In [20]:
num_hiddens = 256
rnn_layer = nn.RNN(len(vocab), num_hiddens)

我们使用张量来初始化隐状态，它的形状是（隐藏层数，批量大小，隐藏单元数）。

In [21]:
# 一层隐藏层，256个隐藏单元
state = torch.zeros(1, batch_size, num_hiddens)
state.shape

torch.Size([1, 32, 256])

通过一个隐状态和一个输入，我们就可以用更新后的隐状态计算输出。
需要强调的是，`rnn_layer`的“输出”（`Y`）不涉及输出层的计算：
它是指每个时间步的隐状态，这些隐状态可以用作后续输出层的输入。

In [22]:
# 时间步数，批量大小，隐藏单元数
X = torch.rand(size=(num_steps, batch_size, len(vocab)))
""" 
Y 是一个形状为 (num_steps, batch_size, num_hiddens) 的张量，表示 RNN 层的输出。
其中，num_steps 是时间步数，batch_size 是批量大小，num_hiddens 是隐藏单元的数量。

state_new 是一个形状为 (1, batch_size, num_hiddens) 的张量，
表示 RNN 层的最新隐藏状态。
其中，1 表示只有一层隐藏层，batch_size 是批量大小，num_hiddens 是隐藏单元的数量。
"""
Y, state_new = rnn_layer(X, state)
Y.shape, state_new.shape

(torch.Size([35, 32, 256]), torch.Size([1, 32, 256]))

我们为一个完整的循环神经网络模型定义了一个`RNNModel`类。
注意，`rnn_layer`只包含隐藏的循环层，我们还需要创建一个单独的输出层。

In [23]:
""" 循环神经网络模型 """
class RNNModel(nn.Module):
    def __init__(self, rnn_layer, vocab_size, **kwargs):
        super(RNNModel, self).__init__(**kwargs)
        self.rnn = rnn_layer
        self.vocab_size = vocab_size
        self.num_hiddens = self.rnn.hidden_size
        # 如果RNN是双向的（之后将介绍），num_directions应该是2，否则应该是1
        if not self.rnn.bidirectional:
            self.num_directions = 1
            self.linear = nn.Linear(self.num_hiddens, self.vocab_size)
        else:
            self.num_directions = 2
            self.linear = nn.Linear(self.num_hiddens * 2, self.vocab_size)

    def forward(self, inputs, state):
        X = F.one_hot(inputs.T.long(),self.vocab_size)
        X = X.to(torch.float32)
        Y, state = self.rnn(X, state)
        # 全连接层首先将Y的形状改为（时间步数*批量大小，隐藏单元数）
        # 输出形状为：（时间步数*批量大小，词表大小）
        # reshape的目的：将Y的每个时间步和批量大小的输出映射到词表大小的维度上
        # -1的作用是根据Y张量的形状和其他维度的大小来自动计算第一个维度的大小，以确保总元素数量保持不变。
        output = self.linear(Y.reshape((-1, Y.shape[-1])))
        return output, state
    
    def begin_state(self, device, batch_size=1):
        if not isinstance(self.rnn, nn.LSTM):
            # nn.GRU以张量作为隐状态
            return torch.zeros((self.num_directions * self.rnn.num_layers,
                                batch_size, self.num_hiddens),
                                device=device)
        else:
            # nn.LSTM以元组作为隐状态
            return (torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),device=device),
                    torch.zeros((self.num_directions * self.rnn.num_layers,
                                 batch_size, self.num_hiddens),device=device))

### 7.6.2 训练与预测
在训练模型之前，让我们基于一个具有随机权重的模型进行预测。

In [24]:
device = d2l.try_gpu()
net = RNNModel(rnn_layer, vocab_size=len(vocab))
net = net.to(device)
d2l.predict_ch8('time traveller', 10, net, vocab, device)

'time travellerwwwwwwwwww'

很明显，这种模型根本不能输出好的结果。 接下来，我们使用定义的超参数调用train_ch8，并且使用高级API训练模型。

In [25]:
num_epochs, lr = 500, 1
d2l.train_ch8(net, train_iter, vocab, lr, num_epochs, device)

perplexity 1.3, 432114.1 tokens/sec on cuda:0
time traveller for so it will spee of the trammels ant sove ly o
travellerifus in any you toust ac ione of time han ut ave i


由于深度学习框架的高级API对代码进行了更多的优化， 该模型在较短的时间内达到了较低的困惑度。

### 7.6.3 小结
- 深度学习框架的高级API提供了循环神经网络层的实现。

- 高级API的循环神经网络层返回一个输出和一个更新后的隐状态，我们还需要计算整个模型的输出层。

- 相比从零开始实现的循环神经网络，使用高级API实现可以加速训练。